# **CASE: ENERGIA**

---

### Instalação de bibliotecas

In [ ]:
# Para teste de estacionariedade
install.packages("urca")
# Para ajuste automático de modelo ARIMA e realização de previsões
install.packages("forecast")

### Carregamento de bibliotecas

In [ ]:
library(urca)
library(forecast)

### Leitura da base de dados

In [ ]:
dados_energia <- read.table(file = "Energia.txt",
                            sep = "\t",
                            dec = ".",
                            header = TRUE)

### Visualização da base de dados

In [ ]:
View(dados_energia)

### Definição das séries

In [ ]:
# Série de consumo (resposta)
serie_consumo <- ts(dados_energia$CONSUMO_GWH)

In [ ]:
# Série de capacidade instalada (explicativa)
serie_capacidade <- ts(dados_energia$CAPACIDADE_INSTALADA_MW)

### Análise exploratória das séries

In [ ]:
# Comando auxiliar para aumentar resolução do gráfico (recomendado apenas no Colab)
reset_options <- options()
options(repr.plot.width = 14, repr.plot.height = 5)

In [ ]:
# Gráficos individuais das séries e gráfico de dispersão
par(mfrow = c(1,3))
plot(serie_consumo,
     type = "l",
     lwd  = 1.5,
     col  = "darkturquoise",
     main = "Consumo de energia (GWh)",
     xlab = "Quinzena",
     ylab = "Índice (GWh)")
plot(serie_capacidade,
     type = "l",
     lwd  = 1.5,
     col  = "darkturquoise",
     main = "Capacidade instalada (MW)",
     xlab = "Quinzena",
     ylab = "Índice (MW)")
plot(dados_energia$CAPACIDADE_INSTALADA_MW,
     dados_energia$CONSUMO_GWH,
     col  = "darkturquoise",
     pch  = 19,
     main = "Consumo de energia (GWh) versus capacidade instalada (MW)",
     xlab = "Capacidade instalada (MW)",
     ylab = "Consumo (GWh)")

In [ ]:
# Teste de estacionariedade de Dickey-Fuller Generalizado (DF-GLS) para série de consumo de energia
teste_consumo <- ur.ers(serie_consumo)
summary(teste_consumo)

In [ ]:
# Verificação de se é possível rejeitar a hipótese nula de não estacionariedade (TRUE representa sim, FALSE representa não)
teste_consumo@teststat < teste_consumo@cval

In [ ]:
# Teste de estacionariedade de Dickey-Fuller Generalizado (DF-GLS) para série de capacidade instalada
teste_capacidade <- ur.ers(serie_capacidade)
summary(teste_capacidade)

In [ ]:
# Verificação de se é possível rejeitar a hipótese nula de não estacionariedade (TRUE representa sim, FALSE representa não)
teste_capacidade@teststat < teste_capacidade@cval

### Modelo da classe (S)ARIMA

In [ ]:
# Ajuste do melhor modelo
modelo_arima <- auto.arima(serie_consumo)
print(modelo_arima)

### Modelo de regressão linear simples

In [ ]:
# Comando auxiliar para para omitir notação científica nos p-valores
options(scipen = 999)

In [ ]:
# Ajuste do modelo
modelo_lin <- lm(serie_consumo ~ serie_capacidade)
summary(modelo_lin)

In [ ]:
# Cálculo dos valores preditos
predito_modelo_lin <- ts(modelo_lin$fitted.values)

### Modelo de regressão linear temporal com erros (S)ARIMA

In [ ]:
# Ajuste do melhor modelo
modelo_lin_arima <- auto.arima(serie_consumo, xreg = serie_capacidade)
print(modelo_lin_arima)

### Comparação visual dos modelos

In [ ]:
# Gráfico dos valores observados da série versus valores preditos pelos dois modelos anteriores
par(mfrow = c(1,1))
plot(serie_consumo,
     type = "l",
     lwd = 1.5,
     col = "grey",
     main = "Comparação dos modelos para o consumo de energia (GWh)",
     xlab = "Quinzena",
     ylab = "Índice (GWh)")
points(modelo_arima$fitted,
       type = "l",
       lty = 3,
       lwd = 3,
       col = "orange")
points(predito_modelo_lin,
       type = "l",
       lty = 3,
       lwd = 3,
       col = "red")
points(modelo_lin_arima$fitted,
       type = "l",
       lwd = 2,
       col = "darkturquoise")

### Diagnóstico dos modelos

In [ ]:
# Cálculo do REQM para modelo da classe (S)ARIMA
sqrt(mean(modelo_arima$residuals^2))

In [ ]:
# Cálculo do REQM para modelo de regressão linear simples
sqrt(mean(modelo_lin$residuals^2))

In [ ]:
# Cálculo do REQM para modelo de regressão linear temporal com erros (S)ARIMA
sqrt(mean(modelo_lin_arima$residuals^2))

### Previsões do melhor modelo





In [ ]:
# Salvando num objeto à parte os índices de capacidade instalada para as próximas duas quinzenas
dados_xreg <- c(200, 230)

In [ ]:
# Previsões para as próximas duas quinzenas
previsoes <- forecast(modelo_lin_arima, h = 2, xreg = dados_xreg)
round(previsoes$mean, 1)

In [ ]:
# Gráfico da série de consumo, incluindo previsões
plot(previsoes,
     type = "l",
     lwd = 2,
     col = "darkturquoise",
     main = "Consumo de energia (GWh)",
     xlab = "Quinzena",
     ylab = "Índice (GWh)")